---
<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2018</h1>
<h1 style="font-size: 250%;">Problem Set #4</h1>
<h1>Issued Monday, 3/26/2018; Due Monday, 4/5/2018</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of	70.71% for each class meeting after the due date. Feel free to get help from others, but the work you submit in should be your own.

In [1]:
# Replace the following string values with the requested information
class Student:
    first = "Mohamed"
    last = "Lansari"
    onyen = "mlans13"
    pid = "730013359"

---
**Problem #1:** Find the longest common subsequence (LCS) between the first 100 bases and the last 100 bases of the [*Salmonella Typhimurium*](http://csbio.unc.edu/mcmillan/Comp555S18/data/SalmonellaTyphimurium.fa) genome. Report both the length of the LCS as well as the subsequence. Do not include the *+* sign at the start of the genome.

In [3]:
# function to load the genome
def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into a list of headers 
        and fragment sequences for each sequence contained"""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'rb')
    else:
        fp = open(filename, 'r')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

# LCS location function
def findLCS(v, w):
    # create our tabulation
    score = [[0 for i in xrange(len(v) + 1)] for j in xrange(len(w) + 1)]
    backtrace = [[0 for i in xrange(len(v) + 1)] for j in xrange(len(w) + 1)]
    
    # iterate through our strings
    for i in xrange(1, len(v) + 1):
        for j in xrange(1, len(w) + 1):
            # checking each intersection, determine right response
            if (v[i - 1] == w[j - 1]):
                score[i][j], backtrace[i][j] = max((score[i - 1][j - 1] + 1, 3), (score[i - 1][j], 1), (score[i][j - 1], 2))
            else:
                score[i][j], backtrace[i][j] = max((score[i - 1][j], 1), (score[i][j - 1], 2))
    return score, backtrace


def extractLCS(b,v,i,j):
    if ((i == 0) and (j == 0)):
        return ''
    if (b[i][j] == 3):
        result = extractLCS(b,v,i-1,j-1)
        result = result + v[i-1]
        return result
    else:
        if (b[i][j] == 2):
            return extractLCS(b,v,i,j-1)
        else:
            return extractLCS(b,v,i-1,j)

# load the fasta
headers, sequences = loadFasta("SalmonellaTyphimurium.fa")
# parse down to the first and last 100 bases
first = sequences[0][1:101]     # be sure to omit the +
last = sequences[0][len(sequences[0]) - 100:len(sequences[0])]

# calculate the least common substring
sc, bt = findLCS(first, last)
# backtrace to read the lcs
res = extractLCS(bt, first, len(bt) - 1, len(bt[0]) - 1)

# print our results
print "The longest common subsequence is: " + res
print "The length of the subsequence is: " + str(len(res))

The longest common subsequence is: GGCTCTGGGAAGAGATAGAAGGGGGTTGTTGAAAAAAAATAACAAATAACGTGGAATTTAATT
The length of the subsequence is: 63


---
**Problem #2:** Find the length of the longest *embedded* common subsequence in the following: 

    "GTACAAAAACAACACAAGAAAAGACAATAAAATTCCATTCGATTCTATTCAATTGGTACA"

The longest embedded common subsequence is the longest common subsequence within in a sequence for which no symbol matches itself. As an example, consider "TAGACATAAG". The longest common subsequence is the sequence itself, "TAGACTAAG", but the longest *embedded* common subsequence is "TAG". 

In [6]:
# LCS location function
def findLECS(v, w):
    # create our tabulation
    score = [[0 for i in xrange(len(v) + 1)] for j in xrange(len(w) + 1)]
    backtrace = [[0 for i in xrange(len(v) + 1)] for j in xrange(len(w) + 1)]
    
    # iterate through our strings
    for i in xrange(1, len(v) + 1):
        for j in xrange(1, len(w) + 1):
            # checking each intersection, determine right response
            if (v[i - 1] == w[j - 1]) and i != j:
                score[i][j], backtrace[i][j] = max((score[i - 1][j - 1] + 1, 3), (score[i - 1][j], 1), (score[i][j - 1], 2))
            else:
                score[i][j], backtrace[i][j] = max((score[i - 1][j], 1), (score[i][j - 1], 2))
    return score, backtrace


def extractLCS(b,v,i,j):
    if ((i == 0) and (j == 0)):
        return ''
    if (b[i][j] == 3):
        result = extractLCS(b,v,i-1,j-1)
        result = result + v[i-1]
        return result
    else:
        if (b[i][j] == 2):
            return extractLCS(b,v,i,j-1)
        else:
            return extractLCS(b,v,i-1,j)

# this is the sampled sequence
test = "GTACAAAAACAACACAAGAAAAGACAATAAAATTCCATTCGATTCTATTCAATTGGTACA"
# run the test
sc, bt = findLECS(test, test)
res = extractLCS(bt, test, len(bt) - 1, len(bt[0]) - 1)

print "The longest embedded subsequence is " + res
print "The length of this is " + str(len(res))

The longest embedded subsequence is AAAAAACACAAAAAAAAAAAAAATTCATTCATTCTTTCA
The length of this is 39


---
**Problem #3:** On your midterm you were asked to find all 16-peptide chains whose molecular weight equals a given target (1024) using the following list of residues ordered by their weights. Most of the algorithms proposed were time-consuming with many possible answers.

```python
Residues = [
    (186, 'W'), (163, 'Y'), (156, 'R'), (147, 'F'), (137, 'H'), 
    (131, 'M'), (129, 'E'), (128, '[K/Q]'), (115, 'D'), (114, 'N'),
    (113, '[I/L]'), (103, 'C'), (101, 'T'), (99, 'V'), (97, 'P'), 
    (87, 'S'), (71, 'A'), (57, 'G')
]
```

Now consider a slightly different problem: find the *length* of the *shortest peptide chain* whose molecular weight matches a target. This problem can be solved effciently using *dynamic programming*. In order to see how, consider an analogy between finding the shortest peptide combination of a given weight, and the problem of making *exact change* from an arbitrary set of coinage. 

In the space below give a reccurance relation for the dynamic program for finding the shortest combination of peptides, *DPShortestPeptideLength(target, Residues)*.

In [ ]:
Residues = [
    (186, 'W'), (163, 'Y'), (156, 'R'), (147, 'F'), (137, 'H'), 
    (131, 'M'), (129, 'E'), (128, '[K/Q]'), (115, 'D'), (114, 'N'),
    (113, '[I/L]'), (103, 'C'), (101, 'T'), (99, 'V'), (97, 'P'), 
    (87, 'S'), (71, 'A'), (57, 'G')
]



---
**Problem #4:** It is inefficient to create and fully populate a table for *DPShortestPeptideLength(target, Residues)* where every molecular weight less than your given target, as you would for computing exact change. Suggest a more efficient means for representing and populating the memoization of intermediate results needed for *DPShortestPeptideLength(target, Residues)*.

**Problem #5: Programming Problem**

Implement a *dynamic programming* solution for *DPShortestPeptideCombination(target, Residues)*, which is similar to the task described in Problem #4, except finds a shortest peptide chain whose molecular weight matches a target. In the space provided below write your function and test it by generating a shortest peptide combination whose molecular weight is 555, 1024, and 2048.

In [3]:
Residues = [
    (186, 'W'), (163, 'Y'), (156, 'R'), (147, 'F'), (137, 'H'), 
    (131, 'M'), (129, 'E'), (128, '[K/Q]'), (115, 'D'), (114, 'N'),
    (113, '[I/L]'), (103, 'C'), (101, 'T'), (99, 'V'), (97, 'P'), 
    (87, 'S'), (71, 'A'), (57, 'G')
]

# this is the main function
# def DPShortestPeptideCombination(target, residues):
#     # start by generating a dictionary for the residues
#     resDict = dict()
#     for r in residues:
#         resDict[r[0]] = r[1]
        
#     # build a tabulation
#     orderedKeys = sorted(resDict.keys())
#     chain = [[0 for i in xrange(len(orderedKeys))]]
    
#     for val in xrange(orderedKeys[0], target + 1):
#         # set the maximum possible peptide length
#         bestLength = val + 1
        
#         for i in xrange(len(orderedKeys)):
#             if val >= orderedKeys[i]:
#                 currChain = [j for j in chain[val - orderedKeys[i]]]
#                 currChain[i] += 1
#                 if (sum(currChain) < bestLength):
#                     chain[val:val] = [currChain]
#                     bestLength = sum(currChain)
                    
#         print str(len(chain)) + " " + str(val)

#     print chain[target - orderedKeys[0]]
                    
#     return ''.join([chain[target][i] * resDict[orderedKeys[i]] for i in xrange(len(orderedKeys))])

In [4]:
Residues = [
    (186, 'W'), (163, 'Y'), (156, 'R'), (147, 'F'), (137, 'H'), 
    (131, 'M'), (129, 'E'), (128, '[K/Q]'), (115, 'D'), (114, 'N'),
    (113, '[I/L]'), (103, 'C'), (101, 'T'), (99, 'V'), (97, 'P'), 
    (87, 'S'), (71, 'A'), (57, 'G')
]

# this is the main function
# def DPShortestPeptideCombination(target, residues):
#     # start by generating a dictionary for the residues
#     resDict = dict()
#     for r in residues:
#         resDict[r[0]] = r[1]
        
#     # build a tabulation
#     orderedKeys = sorted(resDict.keys())
#     chain = [[0 for i in xrange(len(orderedKeys))]]
    
#     for val in xrange(orderedKeys[0], target + 1):
#         # set the maximum possible peptide length
#         bestLength = val + 1
        
#         for i in xrange(len(orderedKeys)):
#             if val >= orderedKeys[i]:
#                 currChain = [j for j in chain[val - orderedKeys[i]]]
#                 currChain[i] += 1
#                 if (sum(currChain) < bestLength):
#                     chain[val:val] = [currChain]
#                     bestLength = sum(currChain)
                    
#         print str(len(chain)) + " " + str(val)

#     print chain[target - orderedKeys[0]]
                    
#     return ''.join([chain[target][i] * resDict[orderedKeys[i]] for i in xrange(len(orderedKeys))])

def DPShortestPeptideCombination(target, residues):
    # generate a dictionary
    resDict = dict()
    for r in residues:
        resDict[r[0]] = r[1]
    
    orderedKeys = sorted([r[0] for r in residues])
    minProteins = [9999 for _ in xrange(target + 2)]
    proteinsUsed = [0 for _ in xrange(target + 2)]
    
    # calculate the chain
    for val in xrange(target + 1):
        count = val
        nextProtein = None
        for p in [v for v in orderedKeys if v <= val]:
            if minProteins[val - p] + 1 < count:
                count = minProteins[val - p] + 1
                nextProtein = p
        minProteins[val] = count
        proteinsUsed[val] = nextProtein
        
    # trace back the chain
    num = target
    proteinChain = []
    while num >= orderedKeys[0]:
        currP = proteinsUsed[num]
        proteinChain.append(resDict[currP])
        num -= currP
        
    return ''.join(proteinChain)
    
print DPShortestPeptideCombination(555, Residues)
print DPShortestPeptideCombination(1024, Residues)
print DPShortestPeptideCombination(2048, Residues)

GRRW
FRYWWW
GMWWWWWWWWWW


Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set